In [1]:
%matplotlib inline
import logging;logging.basicConfig(level=logging.INFO)
import sys
import pandas as pd

sys.path.append("../")
from aif360.datasets import AdultDataset, GermanDataset, CompasDataset
from aif360.metrics import ClassificationMetric

from IPython.display import Markdown, display
from sklearn.metrics import accuracy_score
import tensorflow as tf

In [2]:
## import dataset
dataset_used = "compas" # "adult", "german", "compas"
protected_attribute_used = 2 # 1, 2

if dataset_used == "adult":
    dataset_orig = AdultDataset()
    if protected_attribute_used == 1:
        privileged_groups = [{'sex': 1}]
        unprivileged_groups = [{'sex': 0}]
    else:
        privileged_groups = [{'race': 1}]
        unprivileged_groups = [{'race': 0}]
    
elif dataset_used == "german":
    dataset_orig = GermanDataset()
    if protected_attribute_used == 1:
        privileged_groups = [{'sex': 1}]
        unprivileged_groups = [{'sex': 0}]
    else:
        privileged_groups = [{'age': 1}]
        unprivileged_groups = [{'age': 0}]
    
elif dataset_used == "compas":
    dataset_orig = CompasDataset()
    if protected_attribute_used == 1:
        privileged_groups = [{'sex': 1}]
        unprivileged_groups = [{'sex': 0}]
    else:
        privileged_groups = [{'race': 1}]
        unprivileged_groups = [{'race': 0}]    

In [3]:
import logictensornetworks2 as ltn
import logictensornetworks2.fuzzy_ops as fuzzy_ops
from sklearn.preprocessing import StandardScaler,RobustScaler,MinMaxScaler
from sklearn.model_selection import cross_val_score, KFold
import numpy as np

In [4]:
Not = ltn.Wrapper_ConnectiveOp(fuzzy_ops.Not_Std())
And = ltn.Wrapper_ConnectiveOp(fuzzy_ops.And_Prod())
Or = ltn.Wrapper_ConnectiveOp(fuzzy_ops.Or_ProbSum())
Implies = ltn.Wrapper_ConnectiveOp(fuzzy_ops.Implies_Reichenbach())
Equiv = ltn.Wrapper_ConnectiveOp(fuzzy_ops.Equiv(And,Implies))
Forall = ltn.experimental.Wrapper_AggregationOp(ltn.experimental.Aggreg_pMeanError(p=5))
Exists = ltn.experimental.Wrapper_AggregationOp(ltn.experimental.Aggreg_pMean(p=5))
scale_orig = StandardScaler()

In [5]:
def f(input):
  var_data = ltn.variable("input", input)
  result = D(var_data)
  return result.numpy()

class Predpred(object):
  def __init__(self, oracle):
        self.oracle = oracle
  def predict(self, data):
      var_data = ltn.variable("input", data)
      result = self.oracle(var_data)
      y_test_pred_prob = result.numpy()
      class_thresh = 0.5
      y_test_pred = np.zeros_like(y_test_pred_prob)
      y_test_pred[y_test_pred_prob >= class_thresh] = 1
      y_test_pred[~(y_test_pred_prob >= class_thresh)] = 0
      return y_test_pred

In [6]:
kf = KFold(n_splits=5)
Xs = scale_orig.fit_transform(dataset_orig.features)
ys = dataset_orig.labels.ravel()

In [7]:
D = ltn.Predicate.MLP([401],hidden_layer_sizes=(100,50))
trainable_variables = D.trainable_variables
optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)
formula_aggregator = ltn.fuzzy_ops.Aggreg_pMeanError(p=5)

kv = {}
for k, (train, test) in enumerate(kf.split(Xs, ys)):
    kv["posits{0}".format(k)] = Xs[train][ys[train]==1].astype(np.float32)
    kv["negats{0}".format(k)] = Xs[train][ys[train]==0].astype(np.float32)
    kv["Xtrain{0}".format(k)] = Xs[train].astype(np.float32)
    kv["ytrain{0}".format(k)] = ys[train].astype(np.float32)
    kv["Xtest{0}".format(k)] = Xs[test].astype(np.float32)
    kv["ytest{0}".format(k)] = ys[test].astype(np.float32)
    
    var_posit = ltn.variable("posits",kv["posits{0}".format(k)])
    var_negat = ltn.variable("negats",kv["negats{0}".format(k)])
    oracle = Predpred(D)
    
    @tf.function
    @ltn.domains()
    def axioms():
        axioms = []
        weights = []
        # forall data_A: A(data_A)
        axioms.append(Forall(ltn.bound(var_posit), D(var_posit)))
        # forall data_B: B(data_B)
        axioms.append(Forall(ltn.bound(var_negat), Not(D(var_negat))))
        axioms = tf.stack([tf.squeeze(ax) for ax in axioms])
        sat_level = formula_aggregator(axioms)
        return sat_level, axioms

    for epoch in range(3000):
        with tf.GradientTape() as tape:
            loss = 1. - axioms()[0]
        grads = tape.gradient(loss, trainable_variables)
        optimizer.apply_gradients(zip(grads, trainable_variables))
        if epoch%200 == 0:
            print("Epoch %d: Sat Level %.3f"%(epoch, axioms()[0]),
                  "Epoch %d: Train-Accuracy %.3f"%(epoch, 
                                                   accuracy_score(kv["ytrain{0}".format(k)],
                                                                  oracle.predict(kv["Xtrain{0}".format(k)]))))
    print("Training finished at Epoch %d with Sat Level %.3f"%(epoch, axioms()[0]),"Epoch %d: Test-Accuracy %.3f"%(epoch, accuracy_score(kv["ytest{0}".format(k)], oracle.predict(kv["Xtest{0}".format(k)]))))
    kv["test_acc{0}".format(k)] = accuracy_score(kv["ytest{0}".format(k)], oracle.predict(kv["Xtest{0}".format(k)]))
    
    dataset_orig_test_pred = dataset_orig.subset(test).copy(deepcopy=True)
    dataset_orig_test_pred.labels = oracle.predict(scale_orig.transform(dataset_orig.subset(test).features))
    classified_metric_debiasing_test = ClassificationMetric(dataset_orig.subset(test), 
                                                     dataset_orig_test_pred,
                                                     unprivileged_groups=unprivileged_groups,
                                                     privileged_groups=privileged_groups)
    
    kv["Disparate_impact{0}".format(k)] = classified_metric_debiasing_test.disparate_impact()
    kv["Parity_Difference{0}".format(k)] = classified_metric_debiasing_test.statistical_parity_difference()

Epoch 0: Sat Level 0.390 Epoch 0: Train-Accuracy 0.584
Epoch 200: Sat Level 0.556 Epoch 200: Train-Accuracy 0.817
Epoch 400: Sat Level 0.575 Epoch 400: Train-Accuracy 0.852
Epoch 600: Sat Level 0.580 Epoch 600: Train-Accuracy 0.871
Epoch 800: Sat Level 0.586 Epoch 800: Train-Accuracy 0.875
Epoch 1000: Sat Level 0.592 Epoch 1000: Train-Accuracy 0.888
Epoch 1200: Sat Level 0.596 Epoch 1200: Train-Accuracy 0.895
Epoch 1400: Sat Level 0.600 Epoch 1400: Train-Accuracy 0.883
Epoch 1600: Sat Level 0.599 Epoch 1600: Train-Accuracy 0.903
Epoch 1800: Sat Level 0.609 Epoch 1800: Train-Accuracy 0.907
Epoch 2000: Sat Level 0.609 Epoch 2000: Train-Accuracy 0.913
Epoch 2200: Sat Level 0.611 Epoch 2200: Train-Accuracy 0.915
Epoch 2400: Sat Level 0.609 Epoch 2400: Train-Accuracy 0.913
Epoch 2600: Sat Level 0.614 Epoch 2600: Train-Accuracy 0.918
Epoch 2800: Sat Level 0.617 Epoch 2800: Train-Accuracy 0.923
Training finished at Epoch 2999 with Sat Level 0.616 Epoch 2999: Test-Accuracy 0.611
Epoch 0: Sat L

In [8]:
dataset_orig_test_pred = dataset_orig.copy(deepcopy=True)
dataset_orig_test_pred.labels = oracle.predict(scale_orig.transform(dataset_orig.features))
display(Markdown("#### Model - with bias - classification metrics"))
classified_metric_debiasing_test = ClassificationMetric(dataset_orig, 
                                                 dataset_orig_test_pred,
                                                 unprivileged_groups=unprivileged_groups,
                                                 privileged_groups=privileged_groups)
print("Dataset: Classification accuracy = %f" % classified_metric_debiasing_test.accuracy())
print("Dataset: Disparate impact = %f" % classified_metric_debiasing_test.disparate_impact())
print("Dataset: Parity Difference = %f" % classified_metric_debiasing_test.statistical_parity_difference())

#### Model - with bias - classification metrics

Dataset: Classification accuracy = 0.868494
Dataset: Disparate impact = 0.839907
Dataset: Parity Difference = -0.088203


In [9]:
X_df = pd.DataFrame(Xs,columns=dataset_orig.feature_names)
X_r_preds = f(np.asarray(Xs).astype(np.float32))
X_df['customer_risk_pred'] = X_r_preds
X_female_df = X_df[X_df['race'] == X_df['race'].unique()[0]]
X_male_df = X_df[X_df['race'] == X_df['race'].unique()[1]]
X_female_df['customer class'] = pd.qcut(X_female_df['customer_risk_pred'],5, labels=[0,1,2,3,4])
X_male_df['customer class'] = pd.qcut(X_male_df['customer_risk_pred'],5, labels=[0,1,2,3,4])
X_inp = pd.concat([X_male_df,X_female_df])
class1F = X_inp[X_inp['race'] == X_inp['race'].unique()[0]][X_inp['customer class'] == 0]
class2F = X_inp[X_inp['race'] == X_inp['race'].unique()[0]][X_inp['customer class'] == 1]
class3F = X_inp[X_inp['race'] == X_inp['race'].unique()[0]][X_inp['customer class'] == 2]
class4F = X_inp[X_inp['race'] == X_inp['race'].unique()[0]][X_inp['customer class'] == 3]
class5F = X_inp[X_inp['race'] == X_inp['race'].unique()[0]][X_inp['customer class'] == 4]
class1M = X_inp[X_inp['race'] == X_inp['race'].unique()[1]][X_inp['customer class'] == 0]
class2M = X_inp[X_inp['race'] == X_inp['race'].unique()[1]][X_inp['customer class'] == 1]
class3M = X_inp[X_inp['race'] == X_inp['race'].unique()[1]][X_inp['customer class'] == 2]
class4M = X_inp[X_inp['race'] == X_inp['race'].unique()[1]][X_inp['customer class'] == 3]
class5M = X_inp[X_inp['race'] == X_inp['race'].unique()[1]][X_inp['customer class'] == 4]

C:\Users\Ben\anaconda3\envs\tf2\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\Ben\anaconda3\envs\tf2\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
C:\Users\Ben\anaconda3\envs\tf2\lib\site-packages\ipykernel_launcher.py:9: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  if __name__ == '__main__':
C:\Users\Ben\anaconda3\envs\tf2\lib\site-packages\ipykernel_launcher.py:10: Us

In [10]:
inpclass1f = class1F.iloc[:,:-2].astype(np.float32).to_numpy()
inpclass2f = class2F.iloc[:,:-2].astype(np.float32).to_numpy()
inpclass3f = class3F.iloc[:,:-2].astype(np.float32).to_numpy()
inpclass4f = class4F.iloc[:,:-2].astype(np.float32).to_numpy()
inpclass5f = class5F.iloc[:,:-2].astype(np.float32).to_numpy()

inpclass1m = class1M.iloc[:,:-2].astype(np.float32).to_numpy()
inpclass2m = class2M.iloc[:,:-2].astype(np.float32).to_numpy()
inpclass3m = class3M.iloc[:,:-2].astype(np.float32).to_numpy()
inpclass4m = class4M.iloc[:,:-2].astype(np.float32).to_numpy()
inpclass5m = class5M.iloc[:,:-2].astype(np.float32).to_numpy()

var_class1f = ltn.variable("?class1F",inpclass1f)
var_class2f = ltn.variable("?class2F",inpclass2f)
var_class3f = ltn.variable("?class3F",inpclass3f)
var_class4f = ltn.variable("?class4F",inpclass4f)
var_class5f = ltn.variable("?class5F",inpclass5f)

var_class1m = ltn.variable("?class1M",inpclass1m)
var_class2m = ltn.variable("?class2M",inpclass2m)
var_class3m = ltn.variable("?class3M",inpclass3m)
var_class4m = ltn.variable("?class4M",inpclass4m)
var_class5m = ltn.variable("?class5M",inpclass5m)

In [13]:
D = ltn.Predicate.MLP([401],hidden_layer_sizes=(100,50))
trainable_variables = D.trainable_variables
optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)
formula_aggregator = ltn.fuzzy_ops.Aggreg_pMeanError(p=4)

In [14]:
kv = {}
for k, (train, test) in enumerate(kf.split(Xs, ys)):
    kv["posits{0}".format(k)] = Xs[train][ys[train]==1].astype(np.float32)
    kv["negats{0}".format(k)] = Xs[train][ys[train]==0].astype(np.float32)
    kv["Xtrain{0}".format(k)] = Xs[train].astype(np.float32)
    kv["ytrain{0}".format(k)] = ys[train].astype(np.float32)
    kv["Xtest{0}".format(k)] = Xs[test].astype(np.float32)
    kv["ytest{0}".format(k)] = ys[test].astype(np.float32)
    

    var_posit = ltn.variable("posits",kv["posits{0}".format(k)])
    var_negat = ltn.variable("negats",kv["negats{0}".format(k)])

    @tf.function
    @ltn.domains()
    def axioms():
        axioms = []
        weights = []
        # forall data_A: A(data_A)
        axioms.append(Forall(ltn.bound(var_posit), D(var_posit)))
        weights.append(2.)
        # forall data_B: B(data_B)
        axioms.append(Forall(ltn.bound(var_negat), Not(D(var_negat))))
        weights.append(2.)
        # Equality Constraints
        axioms.append(Forall(ltn.bound(var_class1f,var_class1m), Equiv(D(var_class1f),D(var_class1m)),p=2))
        weights.append(1.)
        axioms.append(Forall(ltn.bound(var_class2f,var_class2m), Equiv(D(var_class2f),D(var_class2m)),p=2))
        weights.append(1.)
        axioms.append(Forall(ltn.bound(var_class3f,var_class3m), Equiv(D(var_class3f),D(var_class3m)),p=2))
        weights.append(1.)
        axioms.append(Forall(ltn.bound(var_class4f,var_class4m), Equiv(D(var_class4f),D(var_class4m)),p=2))
        weights.append(1.)
        axioms.append(Forall(ltn.bound(var_class5f,var_class5m), Equiv(D(var_class5f),D(var_class5m)),p=2))
        weights.append(1.)
        axioms = tf.stack([tf.squeeze(ax) for ax in axioms])
        weights = tf.stack(weights)
        weighted_axioms = weights*axioms
        sat_level = formula_aggregator(weighted_axioms)
        return sat_level, axioms

    for epoch in range(1000):
        with tf.GradientTape() as tape:
            loss = 1. - axioms()[0]
        grads = tape.gradient(loss, trainable_variables)
        optimizer.apply_gradients(zip(grads, trainable_variables))
        if epoch%200 == 0:
            print("Epoch %d: Sat Level %.3f"%(epoch, axioms()[0]),
                  "Epoch %d: Train-Accuracy %.3f"%(epoch, 
                                                   accuracy_score(kv["ytrain{0}".format(k)],
                                                                  oracle.predict(kv["Xtrain{0}".format(k)]))))
    print("Training finished at Epoch %d with Sat Level %.3f"%(epoch, axioms()[0]),"Epoch %d: Test-Accuracy %.3f"%(epoch, accuracy_score(kv["ytest{0}".format(k)], oracle.predict(kv["Xtest{0}".format(k)]))))
    kv["test_acc{0}".format(k)] = accuracy_score(kv["ytest{0}".format(k)], oracle.predict(kv["Xtest{0}".format(k)]))
    
    dataset_orig_test_pred = dataset_orig.subset(test).copy(deepcopy=True)
    dataset_orig_test_pred.labels = oracle.predict(scale_orig.transform(dataset_orig.subset(test).features))
    classified_metric_debiasing_test = ClassificationMetric(dataset_orig.subset(test), 
                                                     dataset_orig_test_pred,
                                                     unprivileged_groups=unprivileged_groups,
                                                     privileged_groups=privileged_groups)
    
    kv["Disparate_impact{0}".format(k)] = classified_metric_debiasing_test.disparate_impact()
    kv["Parity_Difference{0}".format(k)] = classified_metric_debiasing_test.statistical_parity_difference()

Epoch 0: Sat Level 0.570 Epoch 0: Train-Accuracy 0.861
Epoch 200: Sat Level 0.661 Epoch 200: Train-Accuracy 0.861
Epoch 400: Sat Level 0.667 Epoch 400: Train-Accuracy 0.861
Epoch 600: Sat Level 0.668 Epoch 600: Train-Accuracy 0.861
Epoch 800: Sat Level 0.672 Epoch 800: Train-Accuracy 0.861
Training finished at Epoch 999 with Sat Level 0.674 Epoch 999: Test-Accuracy 0.899
Epoch 0: Sat Level 0.630 Epoch 0: Train-Accuracy 0.863
Epoch 200: Sat Level 0.667 Epoch 200: Train-Accuracy 0.863
Epoch 400: Sat Level 0.670 Epoch 400: Train-Accuracy 0.863
Epoch 600: Sat Level 0.673 Epoch 600: Train-Accuracy 0.863
Epoch 800: Sat Level 0.674 Epoch 800: Train-Accuracy 0.863
Training finished at Epoch 999 with Sat Level 0.674 Epoch 999: Test-Accuracy 0.891
Epoch 0: Sat Level 0.628 Epoch 0: Train-Accuracy 0.862
Epoch 200: Sat Level 0.669 Epoch 200: Train-Accuracy 0.862
Epoch 400: Sat Level 0.670 Epoch 400: Train-Accuracy 0.862
Epoch 600: Sat Level 0.672 Epoch 600: Train-Accuracy 0.862
Epoch 800: Sat Level

#### Average Test Accuracy

In [15]:
(kv["test_acc0"]+kv["test_acc1"]+kv["test_acc2"]+kv["test_acc3"]+kv["test_acc4"])/5

0.8684849775422242

#### Average Demographic Parity

In [16]:
(kv["Disparate_impact0"]+kv["Disparate_impact1"]+kv["Disparate_impact2"]+kv["Disparate_impact3"]+kv["Disparate_impact4"])/5

0.8391295536514767

#### Average Disparate Impact

In [17]:
(kv["Parity_Difference0"]+kv["Parity_Difference1"]+kv["Parity_Difference2"]+kv["Parity_Difference3"]+kv["Parity_Difference4"])/5

-0.08904916580745693

In [18]:
dataset_orig_test_pred = dataset_orig.copy(deepcopy=True)
dataset_orig_test_pred.labels = oracle.predict(scale_orig.transform(dataset_orig.features))
display(Markdown("#### Model - without bias - classification metrics"))
classified_metric_debiasing_test = ClassificationMetric(dataset_orig, 
                                                 dataset_orig_test_pred,
                                                 unprivileged_groups=unprivileged_groups,
                                                 privileged_groups=privileged_groups)
print("Dataset: Classification accuracy = %f" % classified_metric_debiasing_test.accuracy())
print("Dataset: Disparate impact = %f" % classified_metric_debiasing_test.disparate_impact())
print("Dataset: Parity Difference = %f" % classified_metric_debiasing_test.statistical_parity_difference())

#### Model - without bias - classification metrics

Dataset: Classification accuracy = 0.868494
Dataset: Disparate impact = 0.839907
Dataset: Parity Difference = -0.088203
